# Try pd.read_html on string only

In [1]:
import pandas as pd

In [2]:
with open('test.html', mode='r') as f:
    raw_html = f.read()
print(raw_html)

<table style="border-collapse: collapse;"><tr style="background-color:lightGrey"><td style="border: 2px solid black;" translatedBlockId="20200310-163448-359-Z7ZAQ3EB">Ingredient / Category name</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-GPLZ1Z8W">Value (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-LCZ9OZVY">Unit (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-SUW1H8M9">Additional text on line (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-KRTVREK9">Icon flag (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-Y232HQNW">Additional text on the bottom (opt)</td></tr><tr><td glossarytab="20191018-164207-236-DZKI00TG" translatedBlockId="20200310-163448-375-V39DJUE9" style="border: 2px solid black;background-color:#A3CEDC">Chicken</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-3

In [3]:
# parse the html string
html_pd = pd.read_html(io=raw_html, header=0)[0]
html_pd

,Ingredient / Category name,Value (opt),Unit (opt),Additional text on line (opt),Icon flag (opt),Additional text on the bottom (opt)
0,Chicken,20.0,%,NaN,NaN,NaN
1,wheat,NaN,NaN,NaN,NaN,NaN
2,dried poultry protein,NaN,NaN,NaN,NaN,NaN
3,corn,NaN,NaN,NaN,NaN,NaN
4,rice,11.0,%,NaN,NaN,NaN
5,animal fats,NaN,NaN,NaN,NaN,NaN
6,dried beet pulp,NaN,NaN,NaN,NaN,NaN
7,digest,NaN,NaN,NaN,NaN,NaN
8,soya meal,NaN,NaN,NaN,NaN,NaN
9,corn protein meal,NaN,NaN,NaN,NaN,NaN


In [105]:
html_pd.shape

(15, 6)

In [4]:
# transform the dataframe to get one row per product
html_pivot_pd = html_pd.pivot(columns='Ingredient / Category name', 
                              values=['Value (opt)', 'Additional text on line (opt)',
                                      'Icon flag (opt)', 'Additional text on the bottom (opt)'])
html_single_row = html_pivot_pd.bfill().iloc[[0]]
# concat columns levels
html_single_row.columns = [' '.join(col).strip().lower() for col in html_single_row.columns.values]
# remove redundant strings
html_single_row.columns = [col.replace('value (opt) ', '% ') for col in html_single_row.columns.values]
# shorten column names
html_single_row.columns = [col.replace('additional text on line (opt)', 'add_info_line') for col in html_single_row.columns.values]
html_single_row.columns = [col.replace('icon flag (opt)', 'icon_flag') for col in html_single_row.columns.values]
html_single_row.columns = [col.replace('additional text on the bottom (opt)', 'add_info_bottom') for col in html_single_row.columns.values]

In [5]:
html_single_row.iloc[0]

% chicken                                                          20.0
% animal fats                                                       NaN
% corn                                                              NaN
% corn protein meal                                                 NaN
% dehydrated chamomile                                              0.5
% digest                                                            NaN
% dried beet pulp                                                   NaN
% dried egg                                                         NaN
% dried poultry protein                                             NaN
% fish oil                                                          1.6
% minerals                                                          NaN
% rice                                                             11.0
% soya meal                                                         NaN
% wheat                                                         

# Mock multi-row dataset
Simulate dataframe with html table in on of its columns.

In [95]:
text1 = raw_html
text2 = raw_html.replace('>1.6<', '>2<').replace('>11<', '>10.6<')
text3 = """<table style="border-collapse: collapse;"><tr style="background-color:lightGrey"><td style="border: 2px solid black;" translatedBlockId="20200310-163448-359-Z7ZAQ3EB">Ingredient / Category name</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-GPLZ1Z8W">Value (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-LCZ9OZVY">Unit (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-SUW1H8M9">Additional text on line (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-KRTVREK9">Icon flag (opt)</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-Y232HQNW">Additional text on the bottom (opt)</td></tr><tr><td glossarytab="20191018-164207-236-DZKI00TG" translatedBlockId="20200310-163448-375-V39DJUE9" style="border: 2px solid black;background-color:#A3CEDC">Chicken</td><td style="border: 2px solid black;" translatedBlockId="20200310-163448-375-MAS6HN4I">20</td></tr></table>"""

In [96]:
# mock a dataset
mocked_pd = pd.DataFrame(data=[('111', text1, len(text1)), ('222', text2, len(text2)), ('333', text3, len(text3))],
                         columns=['product', 'html', 'html_len'])

In [97]:
mocked_pd.head()

,product,html,html_len
0,111,"<table style=""border-collapse: collapse;""><tr ...",10297
1,222,"<table style=""border-collapse: collapse;""><tr ...",10297
2,333,"<table style=""border-collapse: collapse;""><tr ...",1033


In [109]:
# iterate over all rows of the initial df to parse html to separate df with html attriibutes only
for index, row in enumerate(mocked_pd.iterrows()):
    print(index, len(row[1].html))
    html_parsed_pd = pd.read_html(io=row[1].html, header=0)

    html_pivot_pd = html_parsed_pd[0].pivot(columns='Ingredient / Category name',
                                         values=['Value (opt)', 'Additional text on line (opt)',
                                                 'Icon flag (opt)', 'Additional text on the bottom (opt)'])
    
    html_pivot_pd = html_pivot_pd.bfill().iloc[[0]]
    
    if index == 0:
        # create df for html attributes (with data for first row)
        html_attr_pd = html_pivot_pd
        html_attr_pd['product'] = row[1].loc["product"]
        html_attr_pd.set_index(keys='product', inplace=True)

    else:
        # create pandas df with html attributes for a single product
        html_attr_single_row = html_pivot_pd.bfill().iloc[[0]]
        html_attr_single_row['product'] = row[1].loc["product"]
        html_attr_single_row.set_index(keys='product', inplace=True)

        # append the df to dataframe for html attributes
        html_attr_pd = pd.concat(objs=[html_attr_pd, html_attr_single_row])
            

0 10297
1 10297
2 1033


In [110]:
# df with html attributes
html_attr_pd

Value (opt)                                     \
Ingredient / Category name     Chicken animal fats corn corn protein meal   
product                                                                     
111                               20.0         NaN  NaN               NaN   
222                               20.0         NaN  NaN               NaN   
333                               20.0         NaN  NaN               NaN   

                                                                        \
Ingredient / Category name dehydrated chamomile digest dried beet pulp   
product                                                                  
111                                         0.5    NaN             NaN   
222                                         0.5    NaN             NaN   
333                                         NaN    NaN             NaN   

                                                                     ...  \
Ingredient / Category name dried egg dried poultry protein fish oil  ...   
product                                                              ...   
111                              NaN                   NaN      1.6  ...   
222                              NaN                   NaN      2.0  ...   
333                              NaN                   NaN      NaN  ...   

                           Additional text on the bottom (opt)  \
Ingredient / Category name                              digest   
product                                                          
111                                                        NaN   
222                                                        NaN   
333                                                        NaN   

                                                                            \
Ingredient / Category name dried beet pulp dried egg dried poultry protein   
product                                                                      
111                                    NaN       NaN                   NaN   
222                                    NaN       NaN                   NaN   
333                                    NaN       NaN                   NaN   

                                                                                
Ingredient / Category name fish oil minerals rice soya meal wheat wheat gluten  
product                                                                         
111                             NaN      NaN  NaN       NaN   NaN          NaN  
222                             NaN      NaN  NaN       NaN   NaN          NaN  
333                             NaN      NaN  NaN       NaN   NaN          NaN  

[3 rows x 60 columns]

In [111]:
# let's do some cleaning in column naming and get rid of multi index

# concat columns levels
html_attr_pd.columns = [' '.join(col).strip().lower() for col in html_attr_pd.columns.values]
# remove redundant strings
html_attr_pd.columns = [col.replace('value (opt) ', '% ') for col in html_attr_pd.columns.values]
# shorten column names
html_attr_pd.columns = [col.replace('additional text on line (opt)', 'add_info_line') for col in html_attr_pd.columns.values]
html_attr_pd.columns = [col.replace('icon flag (opt)', 'icon_flag') for col in html_attr_pd.columns.values]
html_attr_pd.columns = [col.replace('additional text on the bottom (opt)', 'add_info_bottom') for col in html_attr_pd.columns.values]
#     html_parsed_pd.columns = html_parsed_pd.iloc[0] 
#     html_parsed_pd = html_parsed_pd[1:]

In [116]:
# df with html attributes
pd.set_option('display.max_columns', None) 
html_attr_pd

,% chicken,% animal fats,% corn,% corn protein meal,% dehydrated chamomile,% digest,% dried beet pulp,% dried egg,% dried poultry protein,% fish oil,% minerals,% rice,% soya meal,% wheat,% wheat gluten,add_info_line chicken,add_info_line animal fats,add_info_line corn,add_info_line corn protein meal,add_info_line dehydrated chamomile,add_info_line digest,add_info_line dried beet pulp,add_info_line dried egg,add_info_line dried poultry protein,add_info_line fish oil,add_info_line minerals,add_info_line rice,add_info_line soya meal,add_info_line wheat,add_info_line wheat gluten,icon_flag chicken,icon_flag animal fats,icon_flag corn,icon_flag corn protein meal,icon_flag dehydrated chamomile,icon_flag digest,icon_flag dried beet pulp,icon_flag dried egg,icon_flag dried poultry protein,icon_flag fish oil,icon_flag minerals,icon_flag rice,icon_flag soya meal,icon_flag wheat,icon_flag wheat gluten,add_info_bottom chicken,add_info_bottom animal fats,add_info_bottom corn,add_info_bottom corn protein meal,add_info_bottom dehydrated chamomile,add_info_bottom digest,add_info_bottom dried beet pulp,add_info_bottom dried egg,add_info_bottom dried poultry protein,add_info_bottom fish oil,add_info_bottom minerals,add_info_bottom rice,add_info_bottom soya meal,add_info_bottom wheat,add_info_bottom wheat gluten
product,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
111,20.0,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,1.6,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,* equivalent to 4.5% chamomile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,20.0,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,2.0,NaN,10.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,* equivalent to 4.5% chamomile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# this dataset can be easily merged to the original dataset (or used as separate context - HTML context)